## 최종

In [77]:
from csv import reader
import os
from tqdm import tqdm
from PIL import Image
from psutil import virtual_memory
import numpy as np
import tensorflow as tf
from gc import collect
from sklearn.model_selection import train_test_split
import pandas as pd
from tensorflow.keras.models import load_model
import cv2

In [78]:
# 초성/중성/종성을 결합하는 함수
def combine_consonants(start, middle, final):

    chosung = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ', '']
    jungsung = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ', '']
    jongsung = ['ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ', '']
    
    # 한글 유니코드 계산 (받침이 없는 경우)
    no_final = 44032 + (chosung.index(start) * 21 + jungsung.index(middle)) * 28
    if final != '':
        # 받침이 있다면 종성 부분을 더해주기
        yes_final = no_final + jongsung.index(final) + 1
        # 받침이 있다면 계산된 버전을 도출
        return chr(yes_final)
    # 받침이 없는 경우에는 그대로 도출
    else:
        return chr(no_final)

In [82]:
import numpy as np

def draw_Hangul_consonants(path_to_image, path_to_model):

    # 초성, 중성, 종성 세팅
    chosung = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ', '']
    jungsung = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ', '']
    jongsung = ['ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ', '']

    # 모델 불러오기
    model = tf.keras.models.load_model(path_to_model)
    

    # 이미지 변환
    # 이미지 열기
    image = Image.open(path_to_image)
    # image = save_array_as_image(path_to_image)
    
    image = image.convert('RGB')
    # print(image.size)
    
    # image = np.resize(image)
    # 이미지 재조정
    image = image.resize((90, 90))

    # 넘파이 배열로 이미지 변환
    image_array = np.asarray(image)
    
    # 픽셀값 표준화
    image_array = [image_array.astype('float32') / 255.0]


    # 리스트를 다시 넘파이 배열로 변환 : 이미지만 변환해서 도출
    X = np.array(image_array)
    
    # 모델 예측
    prediction = model.predict(X)

    # 정답이 모여있는 리스트 만들기
    answers = []

    max_index_1 = np.argmax(prediction[0][0])
    max_index_2 = np.argmax(prediction[1][0])
    max_index_3 = np.argmax(prediction[2][0])
    
    # 하나로 묶기 / 초성, 중성, 종성값으로 변환
    temp = [chosung[max_index_1], jungsung[max_index_2], jongsung[max_index_3]]
    
    # 답안지에 넣기
    answers.append(temp)

    # 둘 중 하나의 디렉토리로 지정되도록 코드 구성
    # picked_image = Image.open(path_to_image)

    # 정답과 이미지 보여주기
    # print(f'예측 답안: {combine_consonants(*answers[0])}')

    # display(picked_image)

    return combine_consonants(*answers[0])

In [90]:
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image

# 모델 불러오기
model_YOLO = YOLO('C:/Users/TH_K/Documents/ds_study/00.Fanal_project/01.YOLOv8/data_zip/best.pt')
model_resnet_path = 'C:/Users/TH_K/Documents/ds_study/00.Fanal_project/01.YOLOv8/ResNet_consonants_model_size90.h5'
# 02234424.png


# 영상 파일 경로 설정
video_path = "C:/Users/TH_K/Documents/ds_study/00.Fanal_project/01.YOLOv8/data_zip/20230320_200909.mp4"

# 영상 불러오기
video = cv2.VideoCapture(video_path)

# 영상 프레임 수와 FPS 가져오기
frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
fps = int(video.get(cv2.CAP_PROP_FPS))

# FPS가 0인 경우 1로 설정
if fps == 0:
    fps = 1

# 1초 단위로 이미지 추출하기

for i in range(0, frame_count, fps):
    video.set(cv2.CAP_PROP_POS_FRAMES, i)
    ret, frame = video.read()
    if ret:
        # YOLO 모델로 객체 탐지
        results = model_YOLO(frame, stream=True)

        # 결과 좌표구하기
        for j, r in enumerate(results):
            boxes = r.boxes.xyxy
            frame_with_boxes = frame.copy()
            
            han_ = []

            # boxes가 비어 있는 경우 루프를 건너뛰기.
            # print(boxes)
            
            if len(boxes) == 0:
                continue
            
            boxes= sorted(boxes, key=lambda x:x[0])
            
            for box in boxes:
                x1, y1, x2, y2 = [int(b) for b in box]
                
                cropped_image = frame_with_boxes[y1:y2, x1:x2]

                alpha = 1.5  # 대비를 조절할 배율
                beta = 30  # 밝기를 조절할 값
                adjusted = cv2.convertScaleAbs(cropped_image, alpha=alpha, beta=beta)

                kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
                emphasized = cv2.filter2D(adjusted, -1, kernel)
                
                # alpha1 = 1.0
                # dst1 = np.clip((1+alpha1) * emphasized - 128 * alpha1, 0, 255).astype(np.uint8)
                
                dst = cv2.normalize(emphasized, None, 0, 255, cv2.NORM_MINMAX)

                # plt.figure(figsize=(8,6))
                # plt.imshow(dst)
                # plt.show()
                
                cv2.imwrite('A_test.jpg', dst)

                han_.append(draw_Hangul_consonants('A_test.jpg', model_resnet_path))

            print(*han_)



0: 448x640 3 syllables, 2148.8ms
Speed: 1.3ms preprocess, 2148.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)



연 흐 으


0: 448x640 5 syllables, 2146.5ms
Speed: 3.8ms preprocess, 2146.5ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)



연 흐 양 과 점


0: 448x640 5 syllables, 3960.7ms
Speed: 1.0ms preprocess, 3960.7ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 640)



연 희 양 과 점


0: 448x640 4 syllables, 3120.8ms
Speed: 2.3ms preprocess, 3120.8ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)


흐 양 과 점
